In [53]:
from sympy import symbols, Function, Eq, dsolve, Derivative 

t, k, m, A = symbols('t k m A', real=True)

x1, x2 = symbols("x_1 x_2", cls=Function, real=True)

eqns = [Eq(Derivative(x1(t),t,2), -2*x1(t)+x2(t)), Eq(Derivative(x2(t),t,2), x1(t)-2*x2(t))]
# display(eqns[0])
# display(eqns[1])

sol = dsolve(eqns, [x1(t), x2(t)])
# display(sol[0])
# display(sol[1])

solution = dsolve(eqns, [x1(t), x2(t)], ics={Derivative(x1(t),t,1).subs(t,0): 0, Derivative(x2(t),t,1).subs(t,0):0, x1(0): 1, x2(0): 0})
display(solution[0])
display(solution[1])

Eq(x_1(t), cos(t)/2 + cos(sqrt(3)*t)/2)

Eq(x_2(t), cos(t)/2 - cos(sqrt(3)*t)/2)

In [33]:
from sympy import symbols, Function, Eq, dsolve, Derivative, Matrix

print(sym.__version__)
t, k, m, A = symbols('t k m A', real=True)

x1, x2 = symbols("x_1 x_2", cls=Function, real=True)

matrix = Matrix([[-2, 1], 
                 [1, -2]])
A= (k/m) * matrix
x_vec = Matrix([x1, x2])

RHS = A * x_vec
# LHS=Matrix([Derivative(x1(t), t, 2), Derivative(x2(t),t,2)])
# display(LHS)
# display(RHS)

# display(LHS-RHS)
# eqn = (LHS-RHS).T.tolist()[0]
# display(eqn)

# solution = dsolve(eqn, )


1.12


C:\Users\Michela Conti\AppData\Local\Temp\ipykernel_14804\77425982.py:11: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  x_vec = Matrix([x1, x2])


TypeError: unsupported operand type(s) for *: 'Mul' and 'UndefinedFunction'